<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_SEVIR_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying to run the training code on a small dataset (https://github.com/amazon-science/earth-forecasting-transformer/tree/main/scripts/cuboid_transformer/sevir)

Install dependecies

In [ ]:
!curl 'https://raw.githubusercontent.com/leahandofir/earthformer-inference-experiments/main/prerequisites.sh' -o ./prerequisites.sh
!bash ./prerequisites.sh
%env HOME=/content/earth-forecasting-transformer
%cd ~

In [2]:
%cd scripts/cuboid_transformer/sevir

/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir


In [3]:
#MASTER_ADDR=localhost MASTER_PORT=10001 python train_cuboid_sevir.py --gpus 2 --cfg cfg_sevir.yaml --ckpt_name last.ckpt --save tmp_sevir

In [7]:
from omegaconf import OmegaConf
conf = OmegaConf.load('./cfg_sevir.yaml')
conf.dataset.start_date = "[2019, 1, 1]"
conf.dataset.end_date = "[2019, 12, 1]"
conf.dataset.train_val_split_date = "[2019, 1, 31]"
conf.dataset.train_test_split_date = "[2019, 5, 1]"
OmegaConf.save(config=conf, f='./my_cfg_sevir.yaml')